In [119]:
import pandas as pd
import numpy as numpy
from geotext import GeoText
import re
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import ElementClickInterceptedException

import spacy

import itertools
import requests

import gender_guesser.detector as gender

import warnings
warnings.filterwarnings('ignore')

#### Borial Management

In [2]:
url = "https://borialmanagement.com/"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("h4", {"class": "personName toBold"})

In [3]:
actores = []
for tag in tags:
    actores.append(tag.text)

In [4]:
actores.remove('GONZALO FERNГЃNDEZ')
actores.remove('VГЌCTOR CASTILLO')
actores.remove('ГЃLVARO QUINTANA')
actores.remove('CARLA DГЌAZ')
actores.remove('MARIO GARCГЌA')
actores.remove('SANTI MARГЌN')
actores.remove('ГЃLVARO FONTALBA')

In [5]:
actores2 = []
for actor in actores:
    if len(re.findall("\W", actor)) > 1:
        pass
    else:
        actores2.append(actor)

In [6]:
actores3 = []
for actor in actores2:
    actores3.append(actor.split())

In [7]:
d = gender.Detector()

nombres_por_genero = {"masculino": [], "femenino": []}

for nombre in actores3:
    nombre_completo = nombre[0].capitalize() + nombre[1].capitalize()
    genero = d.get_gender(nombre[0].capitalize())
    if genero == "male":
        nombres_por_genero["masculino"].append(nombre_completo)
    elif genero == "female":
        nombres_por_genero["femenino"].append(nombre_completo)

In [8]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

dicc = {"actor": [], "formacion": []}

for k, v in nombres_por_genero.items():
    if k == "masculino":
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actores/{nombres_por_genero['masculino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                while resultado[i] != "TELEVISIÓN":
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)
    else:
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actrices/{nombres_por_genero['femenino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                while (resultado[i] != "TELEVISIÓN") and (resultado[i] != "TEATRO") and (resultado[i] != "CINE"):
                    print(i)
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [9]:
df_borial = pd.DataFrame(dicc)

In [60]:
def extraer_ciudad(texto):
    return GeoText(texto).cities

In [61]:
df_borial["ciudad"] = df_borial["formacion"].apply(extraer_ciudad)

In [201]:
def nombres(texto):
    patron = r'[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\s[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+'
    return re.findall(patron, texto)

In [151]:
df_borial["nombres"] = df_borial["formacion"].apply(nombres)

In [160]:
df_borial = df_borial.explode("nombres")

In [163]:
df_borial.to_csv("../data/borial_data")

#### Única Representaciones

In [164]:
url = "https://www.unicarepresentaciones.com/Actores"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [165]:
h_unica = []
for tag in tags:
    h_unica.append(tag.text)

In [166]:
url = "https://www.unicarepresentaciones.com/Actrices"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [167]:
m_unica = []
for tag in tags:
    m_unica.append(tag.text)

In [168]:
actores_unica = h_unica+m_unica

In [169]:
h_unica_scrap = []
for actor in (h_unica):
    h_unica_scrap.append(actor.strip().replace(" ", "-").lower())

In [170]:
m_unica_scrap = []
for actriz in (m_unica):
    m_unica_scrap.append(actriz.strip().replace(" ", "-").lower())

In [171]:
unica_scrap = h_unica_scrap+m_unica_scrap

In [172]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {"actor": [], "formacion": []}

for actor in unica_scrap:
    try:
        url = f"https://www.unicarepresentaciones.com/actores-actrices/{actor}"
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        driver.implicitly_wait(10)
        cv = driver.find_element("css selector", "#block-system-main > div > div.ficha-actores > div.ficha-col-1").text
        elementos = cv.split("\n")
        elementos.pop(0)
        elementos = [i.strip() for i in elementos]
        if "Formación" not in elementos:
            pass
        else:
            indice_formacion = elementos.index("Formación") + 1
            lista_formacion = elementos[indice_formacion:]

            dicc["actor"].append(actor)
            dicc["formacion"].append(lista_formacion)
    except NoSuchElementException:
        driver.close()

In [180]:
df_unica = pd.DataFrame(dicc)

In [181]:
df_unica = df_unica.explode('formacion')

In [177]:
df_unica.to_csv("../data/unica_data_raw")

In [183]:
df_unica["ciudad"] = df_unica["formacion"].apply(extraer_ciudad)

In [202]:
df_unica["nombres"] = df_unica["formacion"].apply(nombres)

In [219]:
df_unica = df_unica.explode("nombres")

In [ ]:
df_unic

#### Listado IMDB

In [17]:
listado_actores = actores2 + actores_unica

['DANIEL ALBALADEJO', 'ENRIQUE ASENJO']

In [79]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {"actor": [], "trabajos": []}
conteo = 0
desplazamiento = [2400, 2500, 2600, 2700, 2800, 2900]
for actor in listado_actores:
    try:
        url = "https://www.imdb.com/?ref_=nv_home"
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        driver.implicitly_wait(10)
        element = driver.find_element("css selector", "#suggestion-search").send_keys(actor)
        driver.find_element("css selector", "#react-autowhatever-navSuggestionSearch--item-0 > a > div.sc-fEXmlR.iAJXhE > div.sc-bjfHbI.fOUGiO.searchResult__constTitle").click()
        driver.implicitly_wait(10)
        for i in desplazamiento:
            try:
                driver.execute_script(f"window.scrollTo(0, {i})")
                driver.implicitly_wait(10)
                sleep(5)
                driver.find_element("css selector", "#iconContext-expand-more").click()
                driver.implicitly_wait(10)
                conteo+=1
                dicc["trabajos"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div").text)
                dicc["actor"].append(actor)
                if conteo == len(dicc["trabajos"]):
                    break
                else:
                    pass
            except ElementClickInterceptedException:
                pass
    except NoSuchElementException:
        dicc["trabajos"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div").text)
        dicc["actor"].append(actor)
        driver.close()

AttributeError: 'NoneType' object has no attribute 'click'

In [83]:
df_trabajos = pd.DataFrame(dicc)
df_trabajos

,actor,trabajos
0,DANIEL ALBALADEJO,Alumbramiento\nPsicólogo\nCorto\n2022\nLa noch...
1,ENRIQUE ASENJO,"Reyes contra Santa\nPolicía 1\n2022\nPocos, bu..."
2,DAFNIS BALDUZ,Los renglones torcidos de Dios\nInspector Soto...
3,MARINA BELLATI,Adios Buenos Aires\nMariela Martínez\n2023\nEl...
4,ANA BOKESA,Élite\nRocío (as Ana Bokesa)\nSerie de TV\n202...
5,JORGE CALVO,"Calle de la Humanidad, 8\nDiego Romano, le con..."
6,JORGE CALVO,"Calle de la Humanidad, 8\nDiego Romano, le con..."
7,JACOBO CAMARENA,HIT\nTeo\nSerie de TV\n2021\n10 episodios
8,PEDRO CARVALHO,O Segundo Homem\nRui\n2022\nAmor Amor\nSerafim...
9,PEDRO CARVALHO,O Segundo Homem\nRui\n2022\nAmor Amor\nSerafim...
